# 各个脚本案例

In [32]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
!python script/video_to_images.py --video-path ./dataset/video/city.gif

In [ ]:
!python script/edge_smooth.py --dataset shinkai --image-size 256

In [1]:
!python inference_image.py --type onnx --checkpoint checkpoint/animegan.onnx --src sample/origin --dest sample/anime

Weight loaded, ready to predict
Found 4 images in sample/origin



  0%|          | 0/4 [00:00<?, ?it/s]
Traceback (most recent call last):
  File "inference_image.py", line 30, in <module>
    main(args)
  File "inference_image.py", line 24, in main
    transformer.transform_in_dir(args.src, args.dest)
  File "E:\MSE\deep learning\cartoonize\pytorchanimegan\inference_onnx.py", line 80, in transform_in_dir
    anime_img = self.transform(image)
  File "E:\MSE\deep learning\cartoonize\pytorchanimegan\inference_onnx.py", line 35, in transform
    model_inference = copy.deepcopy(self.model)
  File "D:\software\anaconda3\envs\openvino\lib\copy.py", line 169, in deepcopy
    rv = reductor(4)
TypeError: can't pickle CompiledModel objects


In [70]:
!python inference_video.py --checkpoint checkpoint/generator_shinkai_24.pth --src dataset/video/city.gif --dest dataset/video/test/1.mp4

Weight loaded, ready to predict
288
160
Animation video saved to dataset/video/test/1.mp4


In [36]:
!python create_onnx.py --checkpoint checkpoint/freeze_d_face/generator_shinkai_train_24_finetune_10.pth --output_onnx checkpoint/animegan_freeze_d_face_finetune.onnx

onnx already exist


# 查看checkpoint训练批数

In [88]:
import argparse
parser = argparse.ArgumentParser()
parser.add_argument('--dataset', type=str, default='shinkai')
parser.add_argument('--device', type=str, default='cpu')
args = parser.parse_args(args=[])

In [89]:
from utils.common import load_checkpoint,load_weight
from modeling.anime_ganv2 import Generator
from modeling.anime_ganv2 import Discriminator
import os
def load_checkpoint(model, checkpoint_dir, posfix=''):
    path = os.path.join(checkpoint_dir, f'{model.name}{posfix}.pth')
    return load_weight(model, path)
G = Generator('shinkai')
D = Discriminator(args).to(args.device)

start_e = load_checkpoint(G, 'checkpoint')
test = load_checkpoint(D, 'checkpoint')
start_e

24

# kaggle

In [ ]:
!git clone https://github.com/lyh792127928/pytorch-animegan.git

In [ ]:
!python pytorch-animegan/train.py --device 'cpu' --epochs 20 --save_interval 1 --checkpoint_dir pytorch-animegan/checkpoint --train_photo_path train_photo --data-dir pytorch-animegan/dataset --dataset shinkai --batch 4 --init_epochs 4 --use_sn 

In [ ]:
import os
import zipfile
import datetime

def file2zip(packagePath, zipPath):
    '''
  :param packagePath: 文件夹路径
  :param zipPath: 压缩包路径
  :return:
  '''
    zip = zipfile.ZipFile(zipPath, 'w', zipfile.ZIP_DEFLATED)
    for path, dirNames, fileNames in os.walk(packagePath):
        fpath = path.replace(packagePath, '')
        for name in fileNames:
            fullName = os.path.join(path, name)
            name = fpath + '\\' + name
            zip.write(fullName, name)
    zip.close()


if __name__ == "__main__":
    # 文件夹路径
    packagePath = '/kaggle/working/pytorch-animegan/checkpoint'
    zipPath = '/kaggle/working/output.zip'
    if os.path.exists(zipPath):
        os.remove(zipPath)
    file2zip(packagePath, zipPath)
    print("打包完成")
    print(datetime.datetime.utcnow())

# 移动文件，用于图片批量移动

In [ ]:
import os
import shutil
def file(p):
    p=p
    z=os.listdir(p)
    for i in z:
        
        if not (os.path.isfile(os.path.join(p,i))):
            pp=os.path.join(p,i)
            file(pp)
        else:
            zz=os.listdir(p)
            for k in zz:
                paa.append(os.path.join(p,k))
if __name__ == '__main__':
    p = 'dataset/human_face/origin'
    n_p = 'dataset/human_face/style'
    paa = []
    file(p)
    
    for f in paa:
        try:
            shutil.move(f,n_p)
        except:
            continue
    print('移动完成')

In [24]:
x = torch.randn(1,3,256,256)
#x.size()[1:]
torch.tensor(x.size()[1:]).tolist()

[3, 256, 256]

In [1]:
from utils.common import load_weight
from modeling.anime_ganv2 import Generator
import torch
torch_model = Generator()
load_weight(torch_model,'checkpoint/generator_shinkai.pth')

24

In [ ]:
output_onnx = 'checkpoint/animegan.onnx'
print("==> Exporting model to ONNX format at '{}'".format(output_onnx))
input_names = ["input0"]
output_names = ["output0"]
# 转ONNX 动态输入转换代码
dynamic_axes= {'input0':[0, 2, 3], 'output0':[0,2,3]} #数字0，1等是指张量的维度，表示哪个维度需要动态输入
inputs = torch.randn(1, 3, 256, 256)
onnx_model = torch.onnx.export(torch_model, inputs, output_onnx,input_names=input_names, output_names=output_names,opset_version=11, dynamic_axes=dynamic_axes)

In [1]:
def denormalize_input(images, dtype=None):
    '''
    [-1, 1] -> [0, 255]
    '''
    images = images * 127.5 + 127.5

    if dtype is not None:
        if isinstance(images, torch.Tensor):
            images = images.type(dtype)
        else:
            # numpy.ndarray
            images = images.astype(dtype)

    return images
def normalize_input(images):
    '''
    [0, 255] -> [-1, 1]
    '''
    return images / 127.5 - 1.0

In [2]:
def preprocess_images(images):
    images = images.astype(np.float32)

    # Normalize to [-1, 1]
    images = normalize_input(images)
    #images = torch.from_numpy(images)
    # Add batch dim
    if len(images.shape) == 3:
        images = np.expand_dims(images, 0)
    # channel first
    images = images.transpose(0, 3, 1, 2)

    return images

In [3]:
#openvino
from openvino.runtime import Core
import openvino.runtime as ov
import numpy as np
import cv2
import time
import torch
 
core = Core()
model=r"checkpoint\animegan.onnx"
model = core.compile_model(model=model)
image = cv2.imread('sample/origin/face.jpg')[:,:,::-1]
'''
input_tensor = preprocess_images(image)
results = model.infer_new_request({0: input_tensor})
predictions = next(iter(results.values()))
pred = denormalize_input(predictions.transpose(0, 2, 3, 1)[0],dtype=np.uint8)
'''

'\ninput_tensor = preprocess_images(image)\nresults = model.infer_new_request({0: input_tensor})\npredictions = next(iter(results.values()))\npred = denormalize_input(predictions.transpose(0, 2, 3, 1)[0],dtype=np.uint8)\n'

In [4]:
infer_request = model.create_infer_request()
input_tensor = preprocess_images(image)
input_tensor = ov.Tensor(input_tensor)
infer_request.set_input_tensor(input_tensor)


In [ ]:
infer_request.start_async()
infer_request.wait()
# Get output tensor for model with one output
output = infer_request.get_output_tensor()
output_buffer = output.data
# output_buffer[] - accessing output tensor data
pred = denormalize_input(output_buffer.transpose(0, 2, 3, 1)[0],dtype=np.uint8)


In [5]:
infer_request.outputs

[<Tensor: shape{0, 0, 0, 0} type: f32>]

In [8]:
cv2.imwrite('1.jpg',pred[:,:,::-1])

True

In [ ]:
infer_request.cancel()

In [6]:
input_tensor

<Tensor: shape{1, 3, 411, 430} type: f32>

In [30]:
import os
from tqdm import tqdm
img_dir = 'sample/origin'
dest_dir = 'sample/anime'
files = os.listdir(img_dir)
print(f'Found {len(files)} images in {img_dir}')


Found 4 images in sample/origin


In [ ]:
for fname in tqdm(files):
    image = cv2.imread(os.path.join(img_dir, fname))[:,:,::-1]
    #anime_img = transform(image)
    input_tensor = preprocess_images(image)
    results = model.infer_new_request({0: input_tensor,1:input_tensor})
    predictions = next(iter(results.values()))
    pred = denormalize_input(predictions.transpose(0, 2, 3, 1)[0],dtype=np.uint8)
    anime_img = pred[:,:,::-1]
    ext = fname.split('.')[-1]
    fname = fname.replace(f'.{ext}', '')
    cv2.imwrite(os.path.join(dest_dir, f'{fname}_anime.jpg'), anime_img)